In [ ]:
!pip uninstall torch

In [ ]:
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -i https://download.pytorch.org/whl/cu113/

In [ ]:
## Need to ensure using torch version 1.10 
import torch
import torchvision
import torchaudio
print(torch.__version__, torchvision.__version__, torchaudio.__version__)
print(torch.cuda.get_device_name(0), torch.cuda.get_device_capability(0))
# x=torch.rand(3,3).to(0)
# print(torch.mm(x, torch.inverse(x)))
# print(torch.ops.image.decode_png, torch.ops.torchvision.roi_align)

In [ ]:
import pandas as pd
import keras
import numpy as np
import cvxpy as cp
from numpy import shape
import csv
import matplotlib.pyplot as plt
from scipy.io import loadmat
import copy

import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F
from itertools import accumulate
import cvxpy as cp

use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
SMALL_SIZE = 16
MEDIUM_SIZE = 16
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

### Load in data

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Demand_household.csv')  

In [ ]:
df = df.drop(columns=['Time', 'Weather'])
df.head(5)

In [ ]:
# Assuming same lines from your example
cols_to_norm = ['Temp', 'Dew Point Temp', 'Rel Hum', 'Wind Spd', 'Visibility', 'Stn Press']
df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
df.head(5)

Add in other features

In [ ]:
sLength = len(df['Demand'])

df['weekend'] = pd.Series(np.random.randn(sLength), index=df.index)
df['pre_day'] = pd.Series(np.random.randn(sLength), index=df.index)

df['hour_temp'] = pd.Series(np.random.randn(sLength), index=df.index)
# df['month_temp'] = pd.Series(np.random.randn(sLength), index=df.index)

In [ ]:
for i in range(sLength):
    date = int(np.floor(i/24))
    hour = i%24
    df.at[i, 'Demand'] = df.at[i, 'Demand']/1000 #change demand to Kwh
    df.at[i, 'hour_temp'] = hour*df.iloc[i]['Temp']
    # df.at[i, 'month_temp'] = df.iloc[i]['Month']*df.iloc[i]['Temp']

    if(date%7 == 0 or date%7 == 6):
        df.at[i, 'weekend'] = 1
    else:
        df.at[i, 'weekend'] = 0
    
    if(i<24):
        df.at[i, 'pre_day'] = df.iloc[i]['Demand']
    else:
        df.at[i, 'pre_day'] = df.iloc[i-24]['Demand']

In [ ]:
df1 = df[df['weekend']==0]
df1 = df1.reset_index(drop=True) 
df.head(5)

Test with sklearn package to see initial baseline load forecast performance

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor

In [ ]:
X = df[['Temp', 'Rel Hum', 'hour_temp', 'pre_day', 'Month']] #'month_temp', 

Y = df['Demand']

In [ ]:
df_feature = np.zeros((730, 24*4+1))
df_bs = np.zeros((730, 24))
T = 24

In [ ]:
for i in range(730):
    df_bs[i,:] = df.iloc[i*T:(i+1)*T]['Demand'].to_numpy()
    
    df_feature[i, :] = np.concatenate((df.iloc[i*T:(i+1)*T]['Temp'].to_numpy(),
                                       df.iloc[i*T:(i+1)*T]['Rel Hum'].to_numpy(),
                                       df.iloc[i*T:(i+1)*T]['hour_temp'].to_numpy(),
                                      #  df.iloc[i*T:(i+1)*T]['month_temp'].to_numpy(),
                                       df.iloc[i*T:(i+1)*T]['pre_day'].to_numpy(),
                                       df.iloc[i*T]['Month']), axis=None)

In [ ]:
## correct one data df_feature[10, 46] NaN error
df_feature[10, 46] = 0.86419753
X_train = df_feature[0:300,:]
X_test = df_feature[300:360,:]

In [ ]:
np.std(df_bs)

In [ ]:
y_train = df_bs[0:300,:]
y_test = df_bs[300:360,:]

In [ ]:
regr = MLPRegressor(hidden_layer_sizes=(200, 100, 100), max_iter=500, 
                    validation_fraction=0.2).fit(X_train, y_train)

In [ ]:
regr.fit(X_train,y_train)

In [ ]:
df_pred = regr.predict(X_test)

In [ ]:
mean_squared_error(y_test, df_pred, squared=False)

In [ ]:
t = 20
plt.plot(df_pred[t], label = 'Predicted')
plt.plot(y_test[t], label = 'True')
plt.legend()

In [ ]:
error = df_pred - y_test
error = error.reshape((-1, 1))
plt.hist(error, bins=50)

In [ ]:
np.std(error)

Add in price data

In [ ]:
### load price data
t2 = loadmat('/content/drive/My Drive/Colab Notebooks/t2_data.mat')
DAP = t2['lambda']

N_start = 365*2+31+29+31 #training days
N_end = 365*3+31+29+31 #training days

T = 24
DAP_seg = DAP[N_start*T:N_end*T] #get 2012 NYISO DAP data

### ALL Utils

In [ ]:
## Baseline demand forecast module: 
class LoadForecast(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LoadForecast, self).__init__()

        self.linear1 = nn.Linear(input_dim, 2*hidden_dim)
        self.linear2 = nn.Linear(2*hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, hidden_dim)
        self.linear4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        output = self.linear4(x)
        return output

In [ ]:
class OptLayer(nn.Module):
    def __init__(self, variables, parameters, objective, inequalities, equalities, **cvxpy_opts):
        super().__init__()
        self.variables = variables
        self.parameters = parameters
        self.objective = objective
        self.inequalities = inequalities
        self.equalities = equalities
        self.cvxpy_opts = cvxpy_opts
        
        # create the cvxpy problem with objective, inequalities, equalities
        self.cp_inequalities = [ineq(*variables, *parameters) <= 0 for ineq in inequalities]
        self.cp_equalities = [eq(*variables, *parameters) == 0 for eq in equalities]
        self.problem = cp.Problem(cp.Minimize(objective(*variables, *parameters)), 
                                  self.cp_inequalities + self.cp_equalities)
        
    def forward(self, *batch_params):
        out, J = [], []
        # solve over minibatch by just iterating
        for batch in range(batch_params[0].shape[0]):
            # solve the optimization problem and extract solution + dual variables
            params = [p[batch] for p in batch_params]
            with torch.no_grad():
                for i,p in enumerate(self.parameters):
                    p.value = params[i].double().numpy()
                self.problem.solve(**self.cvxpy_opts)
                z = [torch.tensor(v.value).type_as(params[0]) for v in self.variables]
                lam = [torch.tensor(c.dual_value).type_as(params[0]) for c in self.cp_inequalities]
                nu = [torch.tensor(c.dual_value).type_as(params[0]) for c in self.cp_equalities]

            # convenience routines to "flatten" and "unflatten" (z,lam,nu)
            def vec(z, lam, nu):
                return torch.cat([a.view(-1) for b in [z,lam,nu] for a in b])

            def mat(x):
                sz = [0] + list(accumulate([a.numel() for b in [z,lam,nu] for a in b]))
                val = [x[a:b] for a,b in zip(sz, sz[1:])]
                return ([val[i].view_as(z[i]) for i in range(len(z))],
                        [val[i+len(z)].view_as(lam[i]) for i in range(len(lam))],
                        [val[i+len(z)+len(lam)].view_as(nu[i]) for i in range(len(nu))])

            # computes the KKT residual
            def kkt(z, lam, nu, *params):
                g = [ineq(*z, *params) for ineq in self.inequalities]
                dnu = [eq(*z, *params) for eq in self.equalities]
                L = (self.objective(*z, *params) + 
                     sum((u*v).sum() for u,v in zip(lam,g)) + sum((u*v).sum() for u,v in zip(nu,dnu)))
                dz = autograd.grad(L, z, create_graph=True)
                dlam = [lam[i]*g[i] for i in range(len(lam))]
                return dz, dlam, dnu

            # compute residuals and re-engage autograd tape
            y = vec(z, lam, nu)
            y = y - vec(*kkt([z_.clone().detach().requires_grad_() for z_ in z], lam, nu, *params))

            # compute jacobian and backward hook
            J.append(autograd.functional.jacobian(lambda x: vec(*kkt(*mat(x), *params)), y))
            y.register_hook(lambda grad,b=batch : torch.solve(grad[:,None], J[b].transpose(0,1))[0][:,0])
            
            out.append(mat(y)[0])
        out = [torch.stack(o, dim=0) for o in zip(*out)]
        return out[0] if len(out) == 1 else tuple(out)        

In [ ]:
class PolytopeProjection(nn.Module):
    def __init__(self, d):
        super().__init__()
        # param: alpha - discomfort utility
        # param: N - upperbound
        # param: M - lowerbound
        self.alpha = nn.Parameter(50*torch.ones(1))
        self.N = nn.Parameter(10*torch.ones(1))
        self.M = nn.Parameter(-10*torch.ones(1))
        
        obj = (lambda x, price, alpha, N, M: price@x+0.5*cp.sum_squares(cp.sqrt(alpha)*x) 
               if isinstance(x, cp.Variable) else price@x+0.5*alpha*torch.sum(x**2))
        
        ineq1 = lambda x, price, alpha, N, M: torch.ones(T, dtype=torch.double)@x-N
        ineq2 = lambda x, price, alpha, N, M: M-torch.ones(T, dtype=torch.double)@x
        self.layer = OptLayer([cp.Variable(d)], [cp.Parameter(d), cp.Parameter(1), cp.Parameter(1), cp.Parameter(1)],
                              obj, [ineq1, ineq2], [])
    
    def forward(self, price):
        return self.layer(price, self.alpha.expand(price.shape[0], *self.alpha.shape),
                          self.N.expand(price.shape[0], *self.N.shape),
                          self.M.expand(price.shape[0], *self.M.shape))

In [ ]:
def get_batch(X,Y,M):
    N = len(Y)
    valid_indices = np.array(range(N))
    batch_indices = np.random.choice(valid_indices,size=M,replace=False)
    batch_xs = X[batch_indices,:]
    batch_ys = Y[batch_indices]
    return batch_xs, batch_ys

In [ ]:
def data_loader(alpha_value, upperbound, lowerbound, DAP_seg, df_bs, N=260, T=24):
    
    df_price = np.zeros((N, T))
    df_dr = np.zeros((N, T))
    df_net = np.zeros((N, T))
    index = 0
    for i in range(N):
        price = DAP_seg[i*T:(i+1)*T].reshape(T,)

        alpha = alpha_value
        N = upperbound
        M = lowerbound

        # define the user objective function 
        x = cp.Variable(T)
        f = price@x+0.5*alpha*cp.sum_squares(x)
        cons = [np.ones(T,)@x <=N, np.ones(T,)@x>=M]
        cp.Problem(cp.Minimize(f), cons).solve(verbose=False, eps_abs=1e-8, eps_rel=1e-8)

        df_price[index, :] = price.T
        df_dr[index, :] = x.value 
        df_net[index, :] = df_bs[index,:]+df_dr[index, :]

        index = index+1
    return df_price, df_dr, df_net

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
df_feature = X_train
df_bs = y_train

### Check the OptNet Behavior for Demand Reponse Model Indentification

In [ ]:
## data dimension
N_train = 200
N_test = 60 

input_dim = df_feature.shape[1]
hidden_dim = 100
output_dim = 24
print(input_dim, hidden_dim, output_dim)

In [ ]:
alpha_value = np.random.uniform(10, 30)
bound = np.random.uniform(0, 5)
print('Generating the', i, '-th sample!', 'Bound:', bound, 'alpha_value:', alpha_value)

df_price, df_dr, df_net = data_loader(alpha_value=alpha_value, upperbound=bound, 
                                      lowerbound=-bound, DAP_seg=DAP_seg, df_bs=df_bs)

price_tensor = torch.from_numpy(df_price)
y_tensor = torch.from_numpy(df_dr)

# fit demand response from price
X_train2_price = price_tensor[0:N_train,:]
y_train2 = y_tensor[0:N_train,:]
y_train2_noise = y_train2+3.65*torch.randn(N_train, T) 

# test
X_test_price = price_tensor[N_train:N_train+N_test,:]
y_test = y_tensor[N_train:N_train+N_test,:]

In [ ]:
M = []
M_pred = []
alpha = []
alpha_pred = [] 

## Repeat training for N. times in order to get the mean error and std for the results;
## Default value: N = 1
N = 1
for i in range(N):    
    alpha_value = np.random.uniform(10, 50)
    bound = np.random.uniform(0, 5)
    print('Generating the', i, '-th sample!', 'Bound:', bound, 'alpha_value:', alpha_value)

    df_price, df_dr, df_net = data_loader(alpha_value=alpha_value, upperbound=bound, 
                                          lowerbound=-bound, DAP_seg=DAP_seg, df_bs=df_bs)

    price_tensor = torch.from_numpy(df_price)
    y_tensor = torch.from_numpy(df_dr)

    # fit demand response from price
    X_train2_price = price_tensor[0:N_train,:]
    y_train2 = y_tensor[0:N_train,:]

    # Define model and optimizer
    torch.manual_seed(0)
    layer = PolytopeProjection(d = 24)
    opt1 = optim.Adam(layer.parameters(), lr=5e-1)

    for ite in range(500):
        if(ite == 30):
            opt1.param_groups[0]["lr"] = 1e-1

        dr_pred = layer(X_train2_price)

        loss = nn.MSELoss()(y_train2, dr_pred)
        opt1.zero_grad()
        loss.backward()
        opt1.step()
        
        if(ite%100 == 0):
            print(ite)
            print('Loss', loss.detach())
            print('layer.alpha.gradient =', layer.alpha.grad, 'alpha value =', layer.alpha.detach().numpy())
            print('layer.M.gradient =', layer.M.grad, 'M value =', layer.M.detach().numpy())
            print('layer.N.gradient =', layer.N.grad)

    M.append(-bound)
    M_pred.append(layer.M.detach().numpy())
    alpha.append(alpha_value)
    alpha_pred.append(layer.alpha.detach().numpy())
    
    print('alpha True:', alpha_value, 'Forecast:', layer.alpha.detach().numpy())
    print('M True:', -bound, 'Forecast:', layer.M.detach().numpy())

In [ ]:
# test on test data set
ytest_pred = layer(X_test_price)
ytest_pred_np = ytest_pred.detach().numpy().reshape(N_test*T)
ytest_np = y_test.detach().numpy().reshape(N_test*T)

print('DR forecast Percentage Error', np.mean(abs((ytest_pred_np-ytest_np)/ytest_np)))
print('DR forecast RMSE', np.sqrt(np.sum((ytest_pred_np-ytest_np)**2)/N_test))

In [ ]:
plt.figure(figsize=(6,4))
# plt.plot(demand_sample, 'b', label = 'Original')
plt.plot(ytest_np[0:120], 'g', label = 'DR True')
plt.plot(ytest_pred_np[0:120], 'r--', label = 'DR Forecast')
plt.grid()
plt.legend(loc = 'lower right')
plt.xlabel('Time (h)')
plt.xlim([0, 120])
plt.ylabel('Demand (MW)')
plt.tight_layout()
plt.savefig('DR_5day.png')

### Warm start with end-to-end training

In [ ]:
N_train1 = 200 #pre-train base load module
N_train2 = 200 #e2e training
N_train = 200
N_test = 60 

In [ ]:
# data for baseline demand training
Feature_tensor = torch.from_numpy(df_feature).float()
Baseline_tensor = torch.from_numpy(df_bs).float()

X_train1 = Feature_tensor[0:N_train1,:]
y_train1 = Baseline_tensor[0:N_train1,:]

criterion = nn.MSELoss()

In [ ]:
import time
import warnings
warnings.filterwarnings("ignore") 

# define load forecast network
torch.manual_seed(0)

# load forecast layer
load_net = LoadForecast(input_dim=input_dim, hidden_dim=200, output_dim=output_dim).to(device)

opt2 = optim.Adam(load_net.parameters(), lr=0.001)

In [ ]:
## warm start: baseline load model
for ite in range(600):
    running_loss = 0.0
    batch_xs, batch_ys = get_batch(X_train1, y_train1, M = 64)
    batch_xs = batch_xs.to(device)
    batch_ys = batch_ys.to(device)
    batch_pred = load_net(batch_xs)
    loss = criterion(batch_ys, batch_pred)
    opt2.zero_grad()
    loss.backward()
    opt2.step()
    
    if(ite>250):
        opt2.param_groups[0]["lr"] = 0.0001

    # print statistics
    running_loss += loss.item()
    if ite % 50 == 0:    # print every 2000 mini-batches
        print('[%5d] loss: %.3f' %
              (ite + 1, running_loss / 200))
        running_loss = 0.0

In [ ]:
## make the load forecast module is unbias
# baseline_pred = load_net(X_train1.to(device))
# baseline_pred_bias = np.mean(np.sum(y_train1[0:200].detach().numpy() - baseline_pred[0:200].detach().cpu().numpy(), axis = 1))
# baseline_pred_bias

In [ ]:
M = []
M_pred = []
alpha = []
alpha_pred = [] 

## Repeat training for N. times in order to get the mean error and std for the results;
## Default value: N = 1
N = 1
for i in range(N):
  # data for e2e dr identification
  bound = np.random.uniform(1, 10) #2
  alpha_value = np.random.uniform(10, 50) #20
  print('Generating DR agent model!', 'M_value:', bound, 'alpha_value:', alpha_value)

  baseline_load_per1, baseline_feature_per1 = unison_shuffled_copies(df_bs[0:N_train2], df_feature[0:N_train2])

  df_price1, df_dr1, df_net1 = data_loader(alpha_value=alpha_value, upperbound=bound, 
                                          lowerbound=-bound, DAP_seg=DAP_seg, df_bs=baseline_load_per1, N = N_train2)

  baseline_tensor = torch.from_numpy(baseline_load_per1).float().to(device)
  fea_tensor = torch.from_numpy(baseline_feature_per1).float().to(device)
  price_tensor = torch.from_numpy(df_price1)
  net_tensor = torch.from_numpy(df_net1).to(device)
  dr_tensor = torch.from_numpy(df_dr1).to(device)
  
  #define implicit layer model
  torch.manual_seed(0)
  layer = PolytopeProjection(d = 24)
  opt1 = optim.Adam(layer.parameters(), lr=0.1)
  epst1 = time.time()
  for ite in range(500):
      # define e2e training
      dr_pred = layer(price_tensor)
      baseline_pred = load_net(fea_tensor)
      net_pred = baseline_pred+dr_pred.to(device)
      loss = nn.MSELoss()(net_tensor, net_pred)
      opt1.zero_grad()
      opt2.zero_grad()
      loss.backward()
      opt1.step()
      if(ite%50 == 0):
        opt2.step()

      if(ite == 300):
          opt1.param_groups[0]["lr"] = 5e-2

      if(ite%50 == 0):
          print(ite)
          print('Loss', loss.detach())
          print('layer.alpha.gradient =', layer.alpha.grad, 'alpha value =', layer.alpha.detach().cpu().numpy())
          print('layer.M.gradient =', layer.M.grad, 'M value =', layer.M.detach().cpu().numpy())
  
  epst2 = time.time()
  print('Training time / 500 epoch', epst2-epst1)     
  M.append(-bound)
  M_pred.append(layer.M.detach().cpu().numpy())
  alpha.append(alpha_value)
  alpha_pred.append(layer.alpha.detach().cpu().numpy())

  print('alpha True:', alpha_value, 'Forecast:', layer.alpha.detach().cpu().numpy())
  print('M True:', -bound, 'Forecast:', layer.M.detach().cpu().numpy())

In [ ]:
(epst2-epst1)/500

In [ ]:
## time prediction time
epst3 = time.time()
dr_pred = layer(price_tensor)
baseline_pred = load_net(fea_tensor)
net_pred = baseline_pred+dr_pred.to(device)

epst4 = time.time()
print('Prediction time:', (epst4-epst3)/N_train)